In [ ]:
import math
import os
from typing import List, Tuple, Union

import numpy as np
import numpy.linalg as nl

import matrix
import matshow



In [ ]:
Scalar = Union[int, float, complex]
Vector = List[Scalar]
Matrix = List[Vector]



# 자코비 고유치 알고리듬<br>Jacobi Eigenvalue Algorithm



ref : [[0](https://en.wikipedia.org/wiki/Jacobi_eigenvalue_algorithm)], [[1](https://en.wikipedia.org/wiki/Matrix_similarity)], [[2](https://mathworld.wolfram.com/JacobiTransformation.html)]



해당 알고리듬은 대칭행렬의 모든 고유치와를 고유벡터를 한번에 구할 수 있다.<br>The algorithm can find all eigenvalues and eigenvectors of a symmetric matrix.


행렬의 *상사변환* 의 일종인 *자코비 변환* 을 반복한다.<br>It iterates the *Jacobi transformation*, a *similarity transformation*.



상사변환이란, 어떤 행렬 $A$ 에 행렬 $P$ 와 그 역행렬 $P^{-1}$을 좌우에서 곱해주는 것이다.<br>
The *similarity transformation* is to multiply a matrix $P$ and its inverse $P^{-1}$ to a matrix $A$ on both sides.



$$
B = P^{-1}AP
$$

행렬 $A$ 와 $B$ 의 고유치는 같다.<br>The matrices $A$ and $B$ have the same eigenvalues.



In [ ]:
def initialize_jacobi_method(mat_a:Matrix, b_plot:bool=False) -> Tuple[Matrix, Matrix, int, int]:

    if b_plot:
        remove_all_figure_files()

    n = len(mat_a)
    mat_a0 = matrix.alloc_mat(n, n)

    for i in range(n):
        for j in range(n):
            mat_a0[i][j] = mat_a[i][j]
    mat_x = matrix.get_identity_matrix(n)

    counter = 0

    return mat_a0, mat_x, n, counter



In [ ]:
def remove_all_figure_files(ext:str='png') -> None:
  for filename in os.listdir():
    if os.path.splitext(filename)[-1].lower().endswith(ext.lower()):
      os.remove(filename)



In [ ]:
def search_max_off_diagonal(mat_a0:Matrix, n:int) -> Tuple[float, float, int, int]:
    r = 0
    s = 1
    ars = mat_a0[r][s]
    abs_ars = abs(ars)

    for i in range(n - 1):
        for j in range(i + 1, n):
            aij = abs(mat_a0[i][j])
            if aij > abs_ars:
                r = i
                s = j
                abs_ars = aij
                ars = mat_a0[i][j]

    return abs_ars, ars, r, s



In [ ]:
def calc_theta(ars:float, arr:float, ass:float) -> float:
    theta_rad = 0.5 * math.atan2((2.0 * ars), (arr - ass))
    return theta_rad



In [ ]:
def get_givens_rotation_elements(ars:float, b_verbose:bool, mat_a0:Matrix, r:int, s:int) -> Tuple[float, float, float, float]:
    arr = mat_a0[r][r]
    ass = mat_a0[s][s]
    theta_rad = calc_theta(ars, arr, ass)
    if b_verbose:
        print("theta = %s (deg)" % (theta_rad * 180 / math.pi))
    cos = math.cos(theta_rad)
    sin = math.sin(theta_rad)
    return arr, ass, cos, sin



In [ ]:
def jacobi_rotation(
    ars:float, arr:float, ass:float, cos:float, sin:float,
    mat_a0:Matrix, mat_x:Matrix,
    n:int, r:int, s:int,
):
    for k in range(n):
        if k == r:
            pass
        elif k == s:
            pass
        else:
            akr = mat_a0[k][r]
            aks = mat_a0[k][s]
            mat_a0[r][k] = akr * cos + aks * sin
            mat_a0[s][k] = aks * cos - akr * sin

            mat_a0[k][r] = mat_a0[r][k]
            mat_a0[k][s] = mat_a0[s][k]

        xkr = mat_x[k][r]
        xks = mat_x[k][s]
        mat_x[k][r] = xkr * cos + xks * sin
        mat_x[k][s] = xks * cos - xkr * sin
    mat_a0[r][r] = arr * cos * cos + 2.0 * ars * sin * cos + ass * sin * sin
    mat_a0[s][s] = arr * sin * sin - 2.0 * ars * sin * cos + ass * cos * cos
    mat_a0[r][s] = mat_a0[s][r] = 0.0



In [ ]:
def jacobi_method(mat_a:Matrix, epsilon:float=1e-9, b_verbose:bool=False, b_plot:bool=False) -> Tuple[Matrix, Matrix]:
    mat_a0, mat_x, n, counter = initialize_jacobi_method(mat_a)

    if b_plot:
      abs_ars, ars, r, s = search_max_off_diagonal(mat_a0, n)
      matshow.matshow(counter, abs_ars, r, s, mat_a0, mat_x)

    #########################
    while True:
        abs_ars, ars, r, s = search_max_off_diagonal(mat_a0, n)

        if abs_ars < epsilon:
            break
        if b_verbose:
            print("ars = %s" % ars)
            print("r, s = (%g, %g)" % (r, s))

        arr, ass, cos, sin = get_givens_rotation_elements(ars, b_verbose, mat_a0, r, s)

        jacobi_rotation(ars, arr, ass, cos, sin, mat_a0, mat_x, n, r, s)

        counter += 1

        if b_verbose:
            print("mat_a%03d" % counter)
            matrix.show_mat(mat_a0)
            print("mat_x%03d" % counter)
            matrix.show_mat(mat_x)

        if b_plot:
            matshow.matshow(counter, abs_ars, r, s, mat_a0, mat_x)

    return mat_a0, mat_x


Ref : https://en.wikipedia.org/wiki/Jacobi_eigenvalue_algorithm



In [ ]:
matA = [
    [4, -30, 60, -35],
    [-30, 300, -675, 420],
    [60, -675, 1620, -1050],
    [-35, 420, -1050, 700],
]



In [ ]:
w, v = jacobi_method(matA, b_plot=True)



In [ ]:
w



In [ ]:
v



## Test



In [ ]:
import numpy.testing as nt



In [ ]:
mat_test_A = [
    [4, -30, 60, -35],
    [-30, 300, -675, 420],
    [60, -675, 1620, -1050],
    [-35, 420, -1050, 700],
]



In [ ]:
result_test_w, result_test_v = jacobi_method(mat_test_A)



In [ ]:
nt.assert_array_almost_equal(
    np.array(mat_test_A)@np.array(result_test_v),
    np.diag(np.diag(np.array(result_test_w)))@np.array(result_test_v),
)



In [ ]:
nt.assert_array_almost_equal(np.array(result_v), expected_v)



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

